In [1]:
import tensorflow as tf

import pandas as pd
import codecs
import numpy as np
import math
import csv
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import recall_score, precision_score # New
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
import matplotlib.pyplot as plt
import tensorflow.keras.preprocessing
#from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, Flatten, Add, Multiply, Subtract
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Activation, BatchNormalization
# regularizers
from tensorflow.keras.layers import Conv1D, Conv2D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Dropout, SpatialDropout1D
from scipy import signal
import openpyxl
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from matplotlib import pylab as plt
import matplotlib.dates as mdates
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from numpy.linalg import svd
import gc
from keijzer import *
from tensorflow import keras
from keras.models import Model
from keras.layers import *
from tensorflow.keras.optimizers import Adam,SGD
from keras.layers import Dense, Input, Flatten, Add, concatenate, Dropout, Activation, Multiply, Embedding, Layer, Reshape
from keras.layers import LSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D  
from keras.layers import Conv1D, AveragePooling1D, MaxPooling1D
#from keras.ops import convert_to_tensor, convert_to_numpy
#from keras.utils import plot_model
from keras import activations
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
import math

class LossHistory(tensorflow.keras.callbacks.Callback):  # history = LossHistory()
    def init(self):  # history.init()
        self.losses = []
        # self.accs = []
        self.val_losses = []
        # self.val_accs = []
        self.rmses = []
        self.mses = []
        self.maes = []
        self.mapes = []
        self.val_rmses = []
        self.val_mses = []
        self.val_maes = []
        self.val_mapes = []

    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        # self.accs.append(logs.get('acc'))
        self.val_losses.append(logs.get('val_loss'))
        # self.val_accs.append(logs.get('val_accuracy
        self.rmses.append(logs.get('root_mean_squared_error'))
        self.mses.append(logs.get('mean_squared_error'))
        self.maes.append(logs.get('mean_absolute_error'))
        self.mapes.append(logs.get('mean_absolute_percentage_error'))
        self.val_rmses.append(logs.get('val_root_mean_squared_error'))
        self.val_mses.append(logs.get('val_mean_squared_error'))
        self.val_maes.append(logs.get('val_mean_absolute_error'))
        self.val_mapes.append(logs.get('val_mean_absolute_percentage_error'))



def root_squared_mean_error(y_true, y_pred):
    return K.mean((K.abs(y_pred - y_true))*K.square(y_true-K.mean(y_true)))*100

def mse_mae(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true))+K.mean(K.abs(y_pred - y_true))

from keras.layers.advanced_activations import PReLU

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

In [2]:
import scipy as sc
import os

os.environ["CUDA_VISIBLE_DEVICES"] = '0' # 여러개 사용시 '0,1,2' 식으로 하나의 문자열에 입력
gpus = tf.config.experimental.list_physical_devices('GPU') # 호스트 러나임에 표시되는 GPU 장치 목록 반환

if gpus: # 반환된 GPU 장치 목록이 있다면
    try: # 해당 장치에 대한 메모리 증가 활성화 여부 설정
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e: # try문 실패시에 에러문구 출력
        print(e)

print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
data_path = 'C:/Users/smartgrid_AI/Desktop/windpower_ramprate/ercot_2020.csv'
data_source1 = pd.read_csv(data_path)
dat_source1 = pd.DataFrame(data_source1)
#dat_source1 = dat_source1.iloc[:-3,:]

data_path = 'C:/Users/smartgrid_AI/Desktop/windpower_ramprate/ercot_2021.csv'
data_source2 = pd.read_csv(data_path)
dat_source2 = pd.DataFrame(data_source2)
dat_source2 = dat_source2.iloc[:8761,:9]

data_path = 'C:/Users/smartgrid_AI/Desktop/windpower_ramprate/ercot_2022.csv'
data_source3 = pd.read_csv(data_path)
dat_source3 = pd.DataFrame(data_source3)
dat_source3 = dat_source3.iloc[:8760,:9]

data_path = 'C:/Users/smartgrid_AI/Desktop/windpower_ramprate/ercot_2023.csv'
data_source4 = pd.read_csv(data_path)
dat_source4 = pd.DataFrame(data_source4)
dat_source4 = dat_source4.iloc[:8760,:9]

dat_arr1 = np.array(dat_source1)
dat_arr2 = np.array(dat_source2)
dat_arr3 = np.array(dat_source3)
dat_arr4 = np.array(dat_source4)

dat_arr = np.concatenate([dat_arr1, dat_arr2, dat_arr3, dat_arr4],axis=0)
dat_arr.shape # 8784, 8761, 8760, 8760

dat_source = pd.DataFrame(dat_arr, columns=['Time-Date stamp','Date','ERCOT Load','Total Wind Output','Total Wind Installed','Wind Output, % of Load','Wind Output, % of Installed','1-hr MW change','1-hr % change'])
dat = pd.DataFrame()
dat['MW % change'] = np.array(dat_source.iloc[1:,3], dtype=float)-np.array(dat_source.iloc[0:-1,3], dtype=float)
dat['MW % change'] = np.divide(np.array(dat['MW % change']),np.array(dat_source.iloc[1:,4]))*100
dat['Total Wind Output'] = np.divide(np.array(dat_source.iloc[1:,3]), np.array(dat_source.iloc[1:,4]))

ramp_ratio = 20
load_ratio = 10
year_arr = np.concatenate([np.ones(8784)*2020, np.ones(8760)*2021, np.ones(8760)*2022, np.ones(8760)*2023])
year_df = pd.DataFrame(year_arr, columns=['Year'], dtype=int)
wind_df = pd.concat([year_df, dat], axis=1)
wind_df = wind_df.reset_index(drop=True)

std_scaler = sklearn.preprocessing.MinMaxScaler()
std_scaler_load = sklearn.preprocessing.MinMaxScaler()
std_scaler_ramp = sklearn.preprocessing.MinMaxScaler()

wind_df[['Total Wind Output']] = std_scaler.fit_transform(wind_df[['Total Wind Output']])
wind_df[['MW % change']] = std_scaler_ramp.fit_transform(wind_df[['MW % change']])
wind_df = wind_df.iloc[:-1,:]
ramp_df = pd.DataFrame(wind_df)
ramp_df.columns=['Year', 'Wind Change', 'Wind Output']
ramp_df

,Year,Wind Change,Wind Output
0,2020,0.620197,0.644724
1,2020,0.544580,0.658617
2,2020,0.570751,0.683924
3,2020,0.599594,0.721813
4,2020,0.495246,0.714187
...,...,...,...
35058,2023,0.774643,0.484984
35059,2023,0.735512,0.582157
35060,2023,0.601593,0.620917
35061,2023,0.536583,0.631322


In [4]:
def create_dataset(dataset, timesteps, output_timesteps, leadtime):
    dataX, dataY = [], []
    for i in range(len(dataset) - timesteps -output_timesteps - leadtime - 1):
        a = dataset[i:(i + timesteps), :]
        dataX.append(a)
        dataY.append(dataset[(i + timesteps + leadtime):(i+timesteps+output_timesteps+leadtime), :])
    return np.array(dataX), np.array(dataY)

def svdadd(X, start, end, b_size):
    dec_x = X[start:end, :, :]
    U, S, Vh = svd(dec_x.reshape(b_size, -1), full_matrices=True)

    high_sig = np.matmul(np.matmul(U[:,:high_ind], np.diag(S[0:high_ind])), Vh[:high_ind,:])
    low_sig = X[:dec_num, :, :].reshape(b_size,-1)-high_sig

    rec_x = np.zeros((b_size, b_size, high_ind))
    
    for i in range(high_ind):
        rec_x[:,:,i] = np.matmul((U[:,i]*S[i]).reshape(-1,1), Vh[i,:].reshape(1,-1))    
    return rec_x, low_sig

In [5]:
norm_df = np.array(wind_df.iloc[:,2]).reshape(-1,1)
timesteps = 24*7
output_timesteps = 24
leadtime = 6
num_features = 1
X, Y = create_dataset(norm_df, timesteps, output_timesteps, leadtime)
b_size = timesteps

In [6]:
dec_num = timesteps
high_ind = 35

addX = np.zeros((X.shape[0], dec_num, high_ind+1))

for k in range(X.shape[0]//dec_num):
    rec_x, low_sig = svdadd(X, k*dec_num, (k+1)*dec_num, b_size)
    addX[k*dec_num:k*dec_num+dec_num,:, :-1] = rec_x
    addX[k*dec_num:k*dec_num+dec_num,:, -1] = low_sig
    
rec_x, low_sig = svdadd(X, X.shape[0]-dec_num, X.shape[0], b_size)
addX[X.shape[0]-dec_num:X.shape[0], :, :-1] = rec_x
addX[X.shape[0]-dec_num:X.shape[0],:, -1] = low_sig

addX_det = addX[:, :, :-1]
addX_det = np.sum(addX_det, axis=2)
X = np.concatenate([X, addX], axis=2)
num_features = X.shape[2]

In [7]:
trvaX, teX, trvaY, teY = train_test_split(X, Y, test_size=0.1, shuffle=False)

idx = np.arange(trvaX.shape[0])
np.random.shuffle(idx)
strvaX = trvaX[idx]
strvaY = trvaY[idx]
strvaY = strvaY.reshape(-1, output_timesteps)

trvaY = trvaY.reshape(-1, output_timesteps)
teY = teY.reshape(-1, output_timesteps)

strX, svaX, strY, svaY = train_test_split(strvaX, strvaY, test_size=0.3, shuffle=False)
trX, vaX, trY, vaY = train_test_split(trvaX, trvaY, test_size=0.3, shuffle=False)

trvaX.shape, trvaY.shape, teX.shape, teY.shape

((31377, 168, 37), (31377, 24), (3487, 168, 37), (3487, 24))

In [8]:
def MAEMS(y_true, y_pred):
    return K.mean((K.abs(y_pred - y_true))*K.square(y_true))*100

## SCINet - With SVD With Shuffling

In [9]:
import tensorflow as tf
from tensorflow.keras import layers, models

class Splitting(layers.Layer):
    def __init__(self, name="Splitting"):
        super(Splitting, self).__init__(name=name)

    def call(self, x):
        x_even = x[:, ::2, :]
        x_odd = x[:, 1::2, :]
        return x_even, x_odd
class Interactor(layers.Layer):
    def __init__(self, in_planes, kernel=5, dropout=0.5, groups=1, hidden_size=1, INN=True, name="Interactor"):
        super(Interactor, self).__init__(name=name)
        self.modified = INN
        padding = 'same' if kernel % 2 == 1 else 'causal'
        
        self.P = models.Sequential([
            layers.Conv1D(int(in_planes * hidden_size), kernel, padding=padding, groups=groups, name="P_Conv1"),
            layers.LeakyReLU(alpha=0.01, name="P_LeakyReLU"),
            layers.Dropout(dropout, name="P_Dropout"),
            layers.Conv1D(in_planes, 3, padding=padding, groups=groups, name="P_Conv2"),
            layers.Activation('tanh', name="P_Activation")
        ], name="P_Block")
        
        self.U = models.Sequential([
            layers.Conv1D(int(in_planes * hidden_size), kernel, padding=padding, groups=groups, name="U_Conv1"),
            layers.LeakyReLU(alpha=0.01, name="U_LeakyReLU"),
            layers.Dropout(dropout, name="U_Dropout"),
            layers.Conv1D(in_planes, 3, padding=padding, groups=groups, name="U_Conv2"),
            layers.Activation('tanh', name="U_Activation")
        ], name="U_Block")
        
        self.phi = models.Sequential([
            layers.Conv1D(int(in_planes * hidden_size), kernel, padding=padding, groups=groups, name="phi_Conv1"),
            layers.LeakyReLU(alpha=0.01, name="phi_LeakyReLU"),
            layers.Dropout(dropout, name="phi_Dropout"),
            layers.Conv1D(in_planes, 3, padding=padding, groups=groups, name="phi_Conv2"),
            layers.Activation('tanh', name="phi_Activation")
        ], name="phi_Block")
        
        self.psi = models.Sequential([
            layers.Conv1D(int(in_planes * hidden_size), kernel, padding=padding, groups=groups, name="psi_Conv1"),
            layers.LeakyReLU(alpha=0.01, name="psi_LeakyReLU"),
            layers.Dropout(dropout, name="psi_Dropout"),
            layers.Conv1D(in_planes, 3, padding=padding, groups=groups, name="psi_Conv2"),
            layers.Activation('tanh', name="psi_Activation")
        ], name="psi_Block")
        
        self.split = Splitting()

    def call(self, x):
        x_even, x_odd = self.split(x)
        # Interactor는 각 레이어의 입력 모양이 맞아야 함
        d = tf.math.multiply(x_odd, tf.math.exp(self.phi(x_even)))  # x_even의 shape이 (batch_size, 84, in_planes)
        c = tf.math.multiply(x_even, tf.math.exp(self.psi(x_odd)))  # x_odd의 shape이 (batch_size, 84, in_planes)
        
        if self.modified:
            x_even_update = c + self.U(d)
            x_odd_update = d - self.P(c)
        else:
            x_even_update = c - self.U(d)
            x_odd_update = d + self.P(c)
        
        return x_even_update, x_odd_update


class SCINet_Tree(models.Model):
    def __init__(self, in_planes, current_level, kernel_size, dropout, groups, hidden_size, INN, name="SCINet_Tree"):
        super(SCINet_Tree, self).__init__(name=name)
        self.current_level = current_level
        self.interact = Interactor(in_planes, kernel_size, dropout, groups, hidden_size, INN, name=f"Interactor_Level_{current_level}")

        if current_level > 0:
            self.SCINet_Tree_even = SCINet_Tree(in_planes, current_level-1, kernel_size, dropout, groups, hidden_size, INN, name=f"SCINet_Tree_even_{current_level}")
            self.SCINet_Tree_odd = SCINet_Tree(in_planes, current_level-1, kernel_size, dropout, groups, hidden_size, INN, name=f"SCINet_Tree_odd_{current_level}")

    def zip_up_the_pants(self, even, odd):
        zipped = tf.concat([even[:, i:i+1, :] for i in range(even.shape[1])] +
                           [odd[:, i:i+1, :] for i in range(odd.shape[1])], axis=1)
        return zipped

    def call(self, x):
        x_even_update, x_odd_update = self.interact(x)
        if self.current_level == 0:
            return self.zip_up_the_pants(x_even_update, x_odd_update)
        else:
            return self.zip_up_the_pants(self.SCINet_Tree_even(x_even_update), self.SCINet_Tree_odd(x_odd_update))



class SCINet(models.Model):
    def __init__(self, output_len, input_len, input_dim=1, hid_size=1, num_stacks=1, num_levels=3, kernel=5, dropout=0.5, INN=True, name="SCINet"):
        super(SCINet, self).__init__(name=name)
        self.input_dim = input_dim
        self.output_len = output_len
        self.input_len = input_len
        self.num_stacks = num_stacks

        # 각 스택을 정의
        self.blocks = [SCINet_Tree(input_dim, num_levels-1, kernel, dropout, groups=1, hidden_size=hid_size, INN=INN, name=f"SCINet_Tree_Stack_{i}") for i in range(num_stacks)]

        self.projections = [layers.Conv1D(output_len, 1, padding='same', name=f"Projection_Stack_{i}") for i in range(num_stacks)]
        self.global_pool = layers.GlobalAveragePooling1D()

    def call(self, x):
        res = x
        for i in range(self.num_stacks):
            x = self.blocks[i](x)
            x = self.projections[i](x) 
            res = x  
        return self.global_pool(x) 

In [10]:
output_len = 24        
input_len = 168        
input_dim = trvaX.shape[-1]         
hid_size = 9        
num_stacks = 1
num_levels = 2
kernel_size = 12    
dropout = 0.2         
INN = True           

# 모델 정의
with tf.device('/gpu:0'):
    model = SCINet(output_len, input_len, input_dim, hid_size, num_stacks, num_levels, kernel_size, dropout, INN)
    model.build(input_shape=(None, 168, input_dim))
    model.summary()

Model: "SCINet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
SCINet_Tree_Stack_0 (SCINet_ multiple                  2222220   
_________________________________________________________________
Projection_Stack_0 (Conv1D)  multiple                  912       
_________________________________________________________________
global_average_pooling1d (Gl multiple                  0         
Total params: 2,223,132
Trainable params: 2,223,132
Non-trainable params: 0
_________________________________________________________________


In [11]:
gc.collect()

13045

In [12]:
    model.compile(loss=MAEMS, optimizer='adam', metrics=['mse','mae', MAEMS])
    early_stopping =EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    batch_size = 100
    epochs = 1000
    history = LossHistory()
    history.init()

In [13]:
    %%time
    b_size = 168
    hist = model.fit(strX, strY, epochs=epochs, batch_size=b_size, shuffle=False, validation_data=(svaX, svaY), callbacks=[history, early_stopping])  # , checkpoint

Epoch 1/1000
131/131 [==============================] - 21s 101ms/step - loss: 3.7345 - mse: 0.0973 - mae: 0.2530 - MAEMS: 3.7342 - val_loss: 3.3411 - val_mse: 0.0734 - val_mae: 0.2207 - val_MAEMS: 3.3397
Epoch 2/1000
131/131 [==============================] - 11s 83ms/step - loss: 3.0928 - mse: 0.0807 - mae: 0.2279 - MAEMS: 3.0928 - val_loss: 2.9786 - val_mse: 0.0664 - val_mae: 0.2069 - val_MAEMS: 2.9759
Epoch 3/1000
131/131 [==============================] - 11s 83ms/step - loss: 2.7772 - mse: 0.0686 - mae: 0.2069 - MAEMS: 2.7777 - val_loss: 2.6963 - val_mse: 0.0699 - val_mae: 0.2091 - val_MAEMS: 2.6934
Epoch 4/1000
131/131 [==============================] - 11s 83ms/step - loss: 2.5459 - mse: 0.0597 - mae: 0.1901 - MAEMS: 2.5463 - val_loss: 2.4749 - val_mse: 0.0656 - val_mae: 0.2000 - val_MAEMS: 2.4720
Epoch 5/1000
131/131 [==============================] - 11s 83ms/step - loss: 2.3020 - mse: 0.0510 - mae: 0.1729 - MAEMS: 2.3020 - val_loss: 2.5163 - val_mse: 0.0691 - val_mae: 0.2060

In [14]:
gc.collect()

4594

In [15]:
#model3.save('SCINet', save_format='tf')
testPredict = model.predict(teX, batch_size=b_size)

In [16]:
def npMAEMS(y_true, y_pred):
    return np.mean((abs(y_pred - y_true))*np.square(y_true))*100
def npMAEMD(y_true, y_pred):
    return np.mean((abs(y_pred - y_true))*np.square(y_true-np.mean(y_true)))*100
def npMSE(y_true, y_pred):
    return np.mean(np.square(-y_true+y_pred))
def npMAE(y_true, y_pred):
    return np.mean(abs(-y_true+y_pred))

In [17]:
tePredict = testPredict.reshape(-1)
testY = teY.reshape(-1)
print('Error Test Score > MSE == ', npMSE(testY, tePredict), ' MAE == ', npMAE(testY, tePredict), ' MAEMS == ', npMAEMS(testY, tePredict))

Error Test Score > MSE ==  0.04884789016950677  MAE ==  0.17941999954264826  MAEMS ==  2.7640606336811144


## Without Shuffling

In [18]:
output_len = 24        
input_len = 168        
input_dim = trvaX.shape[-1]         
hid_size = 9        
num_stacks = 1
num_levels = 2
kernel_size = 12    
dropout = 0.2         
INN = True           

# 모델 정의
with tf.device('/gpu:0'):
    model2 = SCINet(output_len, input_len, input_dim, hid_size, num_stacks, num_levels, kernel_size, dropout, INN)
    model2.build(input_shape=(None, 168, input_dim))
    model2.summary()

Model: "SCINet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
SCINet_Tree_Stack_0 (SCINet_ multiple                  2222220   
_________________________________________________________________
Projection_Stack_0 (Conv1D)  multiple                  912       
_________________________________________________________________
global_average_pooling1d_1 ( multiple                  0         
Total params: 2,223,132
Trainable params: 2,223,132
Non-trainable params: 0
_________________________________________________________________


In [19]:
gc.collect()

15524

In [20]:
    model2.compile(loss=MAEMS, optimizer='adam', metrics=['mse','mae', MAEMS])
    early_stopping =EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    batch_size = 100
    epochs = 1000
    history2 = LossHistory()
    history2.init()

In [21]:
    %%time
    hist2 = model2.fit(trX, trY, epochs=epochs, batch_size=b_size, shuffle=False, validation_data=(vaX, vaY), callbacks=[history2, early_stopping])  # , checkpoint

Epoch 1/1000
131/131 [==============================] - 19s 94ms/step - loss: 4.4023 - mse: 0.1090 - mae: 0.2685 - MAEMS: 4.3991 - val_loss: 3.4804 - val_mse: 0.1319 - val_mae: 0.3039 - val_MAEMS: 3.4961
Epoch 2/1000
131/131 [==============================] - 11s 84ms/step - loss: 3.7482 - mse: 0.1021 - mae: 0.2611 - MAEMS: 3.7464 - val_loss: 3.4901 - val_mse: 0.1340 - val_mae: 0.3074 - val_MAEMS: 3.5058
Epoch 3/1000
131/131 [==============================] - 11s 83ms/step - loss: 3.6755 - mse: 0.1009 - mae: 0.2591 - MAEMS: 3.6737 - val_loss: 3.5427 - val_mse: 0.1370 - val_mae: 0.3115 - val_MAEMS: 3.5542
Epoch 4/1000
131/131 [==============================] - 11s 83ms/step - loss: 3.5945 - mse: 0.0983 - mae: 0.2550 - MAEMS: 3.5922 - val_loss: 3.5181 - val_mse: 0.1332 - val_mae: 0.3062 - val_MAEMS: 3.5230
Epoch 5/1000
131/131 [==============================] - 11s 83ms/step - loss: 3.5131 - mse: 0.0956 - mae: 0.2505 - MAEMS: 3.5102 - val_loss: 3.4326 - val_mse: 0.1252 - val_mae: 0.2953 

In [22]:
gc.collect()

4598

In [23]:
testPredict2 = model2.predict(teX, batch_size=b_size)

In [24]:
tePredict2 = testPredict2.reshape(-1)
testY = teY.reshape(-1)
print('Error Test Score > MSE == ', npMSE(testY, tePredict), ' MAE == ', npMAE(testY, tePredict), ' MAEMS == ', npMAEMS(testY, tePredict))
print('Error Test Score > MSE == ', npMSE(testY, tePredict2), ' MAE == ', npMAE(testY, tePredict2), ' MAEMS == ', npMAEMS(testY, tePredict2))

Error Test Score > MSE ==  0.04884789016950677  MAE ==  0.17941999954264826  MAEMS ==  2.7640606336811144
Error Test Score > MSE ==  0.10928829504387161  MAE ==  0.27804073018754466  MAEMS ==  2.657115109660811


In [25]:
from statsmodels.stats.diagnostic import acorr_ljungbox
from scipy.stats import norm

def diebold_mariano_test(forecast1, forecast2, actual, opt): 
    
    if opt==0: # MSE
        e1 = actual-forecast1
        e2 = actual-forecast2
        d = e1**2 - e2**2
    elif opt==1: # MAE
        e1 = abs(actual-forecast1)
        e2 = abs(actual-forecast2)
        d = e1 - e2
    else:
        e1 = np.multiply(abs(actual - forecast1), actual**2)
        e2 = np.multiply(abs(actual - forecast2), actual**2)
        d = e1-e2
    
    # Mean of the loss differentials
    mean_d = np.mean(d)
    
    # Standard deviation of the loss differentials
    std_d = np.std(d, ddof=1)
    
    # Calculate the test statistic
    test_stat = (mean_d / std_d) * np.sqrt(len(d))
    
    # Calculate the p-value using a two-tailed test
    p_value = 2 * (1 - norm.cdf(abs(test_stat)))
    
    return test_stat, p_value

In [26]:
diebold_mariano_test(testPredict, testPredict2, teY, 0), diebold_mariano_test(testPredict, testPredict2, teY, 1), diebold_mariano_test(testPredict, testPredict2, teY, 2)

((-38.08709072810582, 0.0),
 (-35.99164069439573, 0.0),
 (1.6499016454883806, 0.09896305413307172))

In [27]:
np.savetxt('SCINet_prop', testPredict)
np.savetxt('SCINet_woshuffling', testPredict2)